# Data Preparation

## Creazione features Region, Month, Season


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 32 kB/s 
     |████████████████████████████████| 198 kB 38.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=93c580926c0756f56ef7866371243e977115befba1951cae4e1d38c89c28aa3f
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
from pyspark.sql.functions import when, col, isnan, count,  regexp_replace, udf
import numpy as np
import pyspark
from pyspark.sql.types import FloatType, DoubleType
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler,MinMaxScaler, RobustScaler, StandardScaler, MaxAbsScaler, PCA
import pyspark.sql.functions as F
from pyspark.ml.clustering import KMeans, BisectingKMeansModel, KMeansSummary
from  pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import ClusteringEvaluator
from matplotlib.pyplot import figure
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from yellowbrick.cluster import KElbowVisualizer
import matplotlib.pyplot as plt
from pyspark.sql import SQLContext
from pyspark.ml.feature import StringIndexer, VectorIndexer, StandardScaler, VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator, ClusteringEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics, BinaryClassificationMetrics
from pyspark.sql.functions import monotonically_increasing_id 
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
from pyspark import SparkContext
# initialize a new Spark Context to use for the execution of the script
sc = SparkContext(appName="MY-APP-NAME", master="local[*]")

In [ ]:
from pyspark.sql import SQLContext
sqlCtx = SQLContext(sc)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:
# load the dataset

rain_path = 'drive/MyDrive/DDAM/Australia Rain/australia_rain_tomorrow_raw.csv'

In [ ]:
# carico il dataframe
df = sqlCtx.read.load(rain_path, format="csv", sep=",", inferSchema="true", header="true")

In [ ]:
# Creazione colonna Month
df = df.withColumn("Month", F.regexp_extract(col("Date"), r'(\d{1,2})/(\d{1,2})/(\d{4})', 2))

# Creazione colonna Season
df = df.withColumn("Season", .when((df.Month == 12.0) & (df.Month <=2.0),"Winter")\
                               .when((df.Month > 2.0) & (df.Month <=5.0),"Spring") \
                               .when((df.Month > 5.0) & (df.Month <=8.0),"Summer")
                                .otherwise("Fall"))

In [ ]:
print("**************Statistiche per variabile Season ********************")
df.createOrReplaceTempView("tmp")
tot = df.count()
query = sqlCtx.sql("SELECT Season, count(*)/"+str(tot)+" * 100 as percentage FROM tmp GROUP BY Season ORDER BY percentage DESC")
query.show()

**************Statistiche per variabile Season ********************
+------+------------------+
|Season|        percentage|
+------+------------------+
|  Fall| 40.82901408648809|
|Spring|26.300169487949475|
|Summer|25.268473131588753|
|Winter| 7.602343293973683|
+------+------------------+



In [ ]:
print("**************Statistiche per variabile Month ********************")
tot = df.count()
query = sqlCtx.sql("SELECT Month, count(*)/"+str(tot)+" * 100 as percentage FROM tmp GROUP BY Month ORDER BY percentage DESC")
query.show()

**************Statistiche per variabile Month ********************
+-----+------------------+
|Month|        percentage|
+-----+------------------+
|   05|  9.18118332126054|
|   03| 9.167821200762344|
|   01| 9.086945208273264|
|   06|  8.71280583432377|
|   10| 8.301393176879312|
|   07| 8.283811439381687|
|   08| 8.271855857883299|
|   11| 8.060171738411876|
|   09|7.9785924764228895|
|   04| 7.951164965926592|
|   12| 7.602343293973683|
|   02| 7.401911486500742|
+-----+------------------+



In [ ]:
query = sqlCtx.sql("SELECT Month, RainToday, count(*) as cnt FROM tmp GROUP BY Month, RainToday ORDER BY cnt DESC")
query.show(24)

+-----+---------+-----+
|Month|RainToday|  cnt|
+-----+---------+-----+
|   01|       No|10380|
|   03|       No|10097|
|   05|       No|10075|
|   10|       No| 9415|
|   06|       No| 9034|
|   11|       No| 8958|
|   04|       No| 8787|
|   08|       No| 8707|
|   09|       No| 8669|
|   07|       No| 8513|
|   12|       No| 8437|
|   02|       No| 8260|
|   06|      Yes| 3224|
|   07|      Yes| 3147|
|   08|      Yes| 2941|
|   05|      Yes| 2856|
|   03|      Yes| 2788|
|   09|      Yes| 2553|
|   04|      Yes| 2423|
|   01|      Yes| 2417|
|   11|      Yes| 2394|
|   10|      Yes| 2295|
|   12|      Yes| 2266|
|   02|      Yes| 2151|
+-----+---------+-----+
only showing top 24 rows



In [ ]:
query = sqlCtx.sql("SELECT Month, location, count(*) as cnt FROM tmp GROUP BY Month, location ORDER BY location, Month DESC")
query.show(24)

+-----+--------+---+
|Month|location|cnt|
+-----+--------+---+
|   12|Adelaide|239|
|   11|Adelaide|266|
|   10|Adelaide|270|
|   09|Adelaide|263|
|   08|Adelaide|262|
|   07|Adelaide|269|
|   06|Adelaide|258|
|   05|Adelaide|259|
|   04|Adelaide|236|
|   03|Adelaide|270|
|   02|Adelaide|224|
|   01|Adelaide|274|
|   12|  Albany|242|
|   11|  Albany|239|
|   10|  Albany|248|
|   09|  Albany|240|
|   08|  Albany|248|
|   07|  Albany|248|
|   06|  Albany|262|
|   05|  Albany|277|
|   04|  Albany|236|
|   03|  Albany|274|
|   02|  Albany|224|
|   01|  Albany|278|
+-----+--------+---+
only showing top 24 rows



In [ ]:
query = sqlCtx.sql("SELECT Date, location FROM tmp GROUP BY Date, location ORDER BY Date")
query.show(24)

+----------+----------------+
|      Date|        location|
+----------+----------------+
|01/01/2008|        Canberra|
|01/01/2009|        Portland|
|01/01/2009|         Penrith|
|01/01/2009|           Cobar|
|01/01/2009|        Watsonia|
|01/01/2009|     Tuggeranong|
|01/01/2009|   BadgerysCreek|
|01/01/2009|   SydneyAirport|
|01/01/2009|        Canberra|
|01/01/2009|          Sydney|
|01/01/2009|           Moree|
|01/01/2009|          Albury|
|01/01/2009|      Wollongong|
|01/01/2009|      Townsville|
|01/01/2009|         Mildura|
|01/01/2009|      WaggaWagga|
|01/01/2009|     Williamtown|
|01/01/2009|       Melbourne|
|01/01/2009|     MountGinini|
|01/01/2009|        Ballarat|
|01/01/2009|    CoffsHarbour|
|01/01/2009|MelbourneAirport|
|01/01/2009|       NorahHead|
|01/01/2009|          Cairns|
+----------+----------------+
only showing top 24 rows



In [ ]:
query = sqlCtx.sql("SELECT Month, RainTomorrow, count(*) as cnt FROM tmp GROUP BY Month, RainTomorrow ORDER BY cnt DESC")
query.show(25)

+-----+------------+-----+
|Month|RainTomorrow|  cnt|
+-----+------------+-----+
|   01|          No|10423|
|   03|          No|10255|
|   05|          No|10114|
|   10|          No| 9494|
|   06|          No| 9145|
|   11|          No| 9025|
|   04|          No| 8843|
|   08|          No| 8792|
|   09|          No| 8739|
|   07|          No| 8608|
|   12|          No| 8560|
|   02|          No| 8318|
|   06|         Yes| 3244|
|   07|         Yes| 3171|
|   08|         Yes| 2970|
|   05|         Yes| 2941|
|   03|         Yes| 2781|
|   09|         Yes| 2606|
|   01|         Yes| 2498|
|   04|         Yes| 2463|
|   11|         Yes| 2436|
|   10|         Yes| 2310|
|   12|         Yes| 2250|
|   02|         Yes| 2207|
+-----+------------+-----+



In [ ]:
query = sqlCtx.sql("SELECT Season, RainTomorrow, count(*) as cnt FROM tmp GROUP BY Season, RainTomorrow ORDER BY cnt DESC")
query.show()

+------+------------+-----+
|Season|RainTomorrow|  cnt|
+------+------------+-----+
|  Fall|          No|45999|
|Spring|          No|29212|
|Summer|          No|26545|
|  Fall|         Yes|12057|
|Summer|         Yes| 9385|
|Winter|          No| 8560|
|Spring|         Yes| 8185|
|Winter|         Yes| 2250|
+------+------------+-----+



In [ ]:
query = sqlCtx.sql("SELECT Season, RainToday, count(*) as cnt FROM tmp GROUP BY Season, RainToday ORDER BY cnt DESC")
query.show()

+------+---------+-----+
|Season|RainToday|  cnt|
+------+---------+-----+
|  Fall|       No|45682|
|Spring|       No|28959|
|Summer|       No|26254|
|  Fall|      Yes|11810|
|Summer|      Yes| 9312|
|Winter|       No| 8437|
|Spring|      Yes| 8067|
|Winter|      Yes| 2266|
|  Fall|       NA|  564|
|Spring|       NA|  371|
|Summer|       NA|  364|
|Winter|       NA|  107|
+------+---------+-----+



In [ ]:
df.show()

+----------+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+-------+------------+-----+------+
|      Date|Location|MinTemp|MaxTemp|Rainfall|Evaporation|Sunshine|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|Cloud9am|Cloud3pm|Temp9am|Temp3pm|RainToday|RISK_MM|RainTomorrow|Month|Season|
+----------+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+-------+------------+-----+------+
|01/12/2008|  Albury|   13.4|   22.9|     0.6|         NA|      NA|          W|           44|         W|       WNW|          20|          24|         71|         22|     1007.7|     1007.1|       

In [ ]:
# Creazione colonna Region

# Percorso file con associazioni città-regione
filename_cities = 'drive/MyDrive/DDAM/Australia Rain/Mario/cities_australia.csv'

# Creazione rdd (problemi usando il dataframe)
cities_rdd = sc.textFile(filename_cities)
cities_rdd.take(3)

['City;Region', 'Melbourne;Victoria', 'Sydney;New South Wales']

In [ ]:
# create a dictionary (k: location, v: region)

cities_rdd_dict = cities_rdd.map(lambda line: line.strip().split(";")).collectAsMap()

cities_rdd_dict['Sydney']

'New South Wales'

In [ ]:
func_name = udf(
    lambda val: cities_rdd_dict[val], 
    StringType()
)

df = df.withColumn('Region', func_name(df.Location))

In [ ]:
df.show(10)

+----------+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+-------+------------+-----+------+---------------+
|      Date|Location|MinTemp|MaxTemp|Rainfall|Evaporation|Sunshine|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|Cloud9am|Cloud3pm|Temp9am|Temp3pm|RainToday|RISK_MM|RainTomorrow|Month|Season|         Region|
+----------+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+-------+------------+-----+------+---------------+
|01/12/2008|  Albury|   13.4|   22.9|     0.6|         NA|      NA|          W|           44|         W|       WNW|          20|          24|       

In [ ]:
df.groupBy('Region').count().show()

+------------------+-----+
|            Region|count|
+------------------+-----+
|     NorfolkIsland| 2964|
|          Victoria|28020|
|   New South Wales|50710|
|        Queensland|12162|
|Northern Territory| 9303|
|   South Australia|12112|
| Western Australia|20706|
|          Tasmania| 6216|
+------------------+-----+



In [ ]:
print("**************Statistiche per variabile Region ********************")
df.createOrReplaceTempView("tmp")
tot = df.count()
query = sqlCtx.sql("SELECT Region, count(*)/"+str(tot)+" * 100 as percentage FROM tmp GROUP BY Region ORDER BY percentage DESC")
query.show()

**************Statistiche per variabile Region ********************
+------------------+------------------+
|            Region|        percentage|
+------------------+------------------+
|   New South Wales| 35.66279634018552|
|          Victoria| 19.70561138733974|
| Western Australia|14.561898265034143|
|        Queensland| 8.553163657845323|
|   South Australia|  8.51800018285007|
|Northern Territory|6.5425161576167605|
|          Tasmania| 4.371523211409844|
|     NorfolkIsland| 2.084490797718594|
+------------------+------------------+



## Tentativi di sostituzione missing values (groupBy)

In [ ]:
# Sostituisco 'NA' con null values 
df = df.replace('NA', None)
df.show(5)

+----------+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+-------+------------+-----+------+---------------+
|      Date|Location|MinTemp|MaxTemp|Rainfall|Evaporation|Sunshine|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|Cloud9am|Cloud3pm|Temp9am|Temp3pm|RainToday|RISK_MM|RainTomorrow|Month|Season|         Region|
+----------+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+-------+------------+-----+------+---------------+
|01/12/2008|  Albury|   13.4|   22.9|     0.6|       null|    null|          W|           44|         W|       WNW|          20|          24|       

In [ ]:
# Colonne con più null values

print('Totale osservazioni: 142193')
print()

Dict_Null = {col:df.filter(df[col].isNull()).count() for col in df.columns}
Dict_Null = sorted(Dict_Null.items(), key=lambda x: x[1], reverse=True)

for i in Dict_Null:
	print(i[0], i[1])

Totale osservazioni: 142193

Sunshine 67816
Evaporation 60843
Cloud3pm 57094
Cloud9am 53657
Pressure9am 14014
Pressure3pm 13981
WindDir9am 10013
WindGustDir 9330
WindGustSpeed 9270
WindDir3pm 3778
Humidity3pm 3610
Temp3pm 2726
WindSpeed3pm 2630
Humidity9am 1774
Rainfall 1406
RainToday 1406
WindSpeed9am 1348
Temp9am 904
MinTemp 637
MaxTemp 322
Date 0
Location 0
RISK_MM 0
RainTomorrow 0
Month 0
Season 0
Region 0


In [ ]:
# Analisi dei null values sulle colonne con più valori mancanti, raggruppati per città

from pyspark.ml.stat import Correlation

cols = ["Evaporation", "Sunshine", "Cloud3pm", "Cloud9am"]

for c in cols:
    x = df.filter(df[c].isNull()).groupby(df.Location).count()

    print('Null values for column {}'.format(c),'(grouped by city)')
    x.sort("count", ascending=False).show(10)  # con print dà un None di troppo

Null values for column Evaporation (grouped by city)
+-----------+-----+
|   Location|count|
+-----------+-----+
|   Ballarat| 3028|
|     Albury| 3011|
|Tuggeranong| 2998|
| Wollongong| 2983|
|  GoldCoast| 2980|
|    Penrith| 2964|
| SalmonGums| 2955|
|  Newcastle| 2955|
|Witchcliffe| 2952|
|  NorahHead| 2929|
+-----------+-----+
only showing top 10 rows

Null values for column Sunshine (grouped by city)
+-----------+-----+
|   Location|count|
+-----------+-----+
|    Bendigo| 3034|
| Launceston| 3028|
|   Ballarat| 3028|
|     Albury| 3011|
|Tuggeranong| 2998|
| Wollongong| 2983|
|  GoldCoast| 2980|
|    Penrith| 2964|
|  Newcastle| 2955|
| SalmonGums| 2955|
+-----------+-----+
only showing top 10 rows

Null values for column Cloud3pm (grouped by city)
+-------------+-----+
|     Location|count|
+-------------+-----+
|     Adelaide| 3090|
|  Tuggeranong| 2998|
|    GoldCoast| 2980|
|      Penrith| 2964|
|   SalmonGums| 2955|
|  Witchcliffe| 2952|
|     Dartmoor| 2943|
|    NorahHead|

In [ ]:
# Analisi dei null values sulle colonne con più valori mancanti, raggruppati per RainTomorrow

cols = ["Evaporation", "Sunshine", "Cloud3pm", "Cloud9am"]

for c in cols:
    x = df.filter(df[c].isNull()).groupby(df.RainTomorrow).count()

    print('Null values for column {}'.format(c),'(grouped by RainTomorrow)')
    x.sort("count", ascending=False).show(10)

Null values for column Evaporation (grouped by RainTomorrow)
+------------+-----+
|RainTomorrow|count|
+------------+-----+
|          No|46825|
|         Yes|14018|
+------------+-----+

Null values for column Sunshine (grouped by RainTomorrow)
+------------+-----+
|RainTomorrow|count|
+------------+-----+
|          No|52760|
|         Yes|15056|
+------------+-----+

Null values for column Cloud3pm (grouped by RainTomorrow)
+------------+-----+
|RainTomorrow|count|
+------------+-----+
|          No|45505|
|         Yes|11589|
+------------+-----+

Null values for column Cloud9am (grouped by RainTomorrow)
+------------+-----+
|RainTomorrow|count|
+------------+-----+
|          No|42402|
|         Yes|11255|
+------------+-----+



In [ ]:
# Analisi dei null values sulle colonne con più valori mancanti, raggruppati per RainToday
# L'attributo RainToday, a differenza dei precedenti, presenta null values

cols = ["Evaporation", "Sunshine", "Cloud3pm", "Cloud9am"]

for c in cols:
    x = df.filter(df[c].isNull()).groupby(df.RainToday).count()

    print('Null values for column {}'.format(c),'(grouped by RainToday)')
    x.sort("count", ascending=False).show(10) 

Null values for column Evaporation (grouped by RainToday)
+---------+-----+
|RainToday|count|
+---------+-----+
|       No|46035|
|      Yes|13659|
|     null| 1149|
+---------+-----+

Null values for column Sunshine (grouped by RainToday)
+---------+-----+
|RainToday|count|
+---------+-----+
|       No|52046|
|      Yes|14759|
|     null| 1011|
+---------+-----+

Null values for column Cloud3pm (grouped by RainToday)
+---------+-----+
|RainToday|count|
+---------+-----+
|       No|44717|
|      Yes|11377|
|     null| 1000|
+---------+-----+

Null values for column Cloud9am (grouped by RainToday)
+---------+-----+
|RainToday|count|
+---------+-----+
|       No|42010|
|      Yes|10615|
|     null| 1032|
+---------+-----+



In [ ]:
# Visualizzo la distribuzione di Yes e No in RainToday e RainTomorrow

df.groupBy("RainToday").count().show()
df.groupBy("RainTomorrow").count().show()

+---------+------+
|RainToday| count|
+---------+------+
|       No|109332|
|      Yes| 31455|
|     null|  1406|
+---------+------+

+------------+------+
|RainTomorrow| count|
+------------+------+
|          No|110316|
|         Yes| 31877|
+------------+------+



In [ ]:
# Media dell'attributo Evaporation (groupby RainToday e RainTomorrow)

df.groupBy("RainToday").agg({'Evaporation':'avg'}).show()
df.groupBy("RainTomorrow").agg({'Evaporation':'avg'}).show()

+---------+-----------------+
|RainToday| avg(Evaporation)|
+---------+-----------------+
|       No|5.890048501508775|
|      Yes|3.987429759496509|
|     null|4.620622568093383|
+---------+-----------------+

+------------+-----------------+
|RainTomorrow| avg(Evaporation)|
+------------+-----------------+
|          No|5.734806508009009|
|         Yes|4.527778711014043|
+------------+-----------------+



In [ ]:
# Media dell'attributo Sunshine (groupby RainToday e RainTomorrow)

df.groupBy("RainToday").agg({'Sunshine':'avg'}).show()
df.groupBy("RainTomorrow").agg({'Sunshine':'avg'}).show()

+---------+-----------------+
|RainToday|    avg(Sunshine)|
+---------+-----------------+
|       No| 8.30556156827152|
|      Yes|5.314458552946793|
|     null|6.559746835443038|
+---------+-----------------+

+------------+-----------------+
|RainTomorrow|    avg(Sunshine)|
+------------+-----------------+
|          No|8.546358329279336|
|         Yes|4.471761488615393|
+------------+-----------------+



In [ ]:
# Media dell'attributo Cloud9am (groupby RainToday e RainTomorrow)

df.groupBy("RainToday").agg({'Cloud9am':'avg'}).show()
df.groupBy("RainTomorrow").agg({'Cloud9am':'avg'}).show()

+---------+-----------------+
|RainToday|    avg(Cloud9am)|
+---------+-----------------+
|       No|3.939796797480764|
|      Yes|6.018474088291747|
|     null|5.858288770053476|
+---------+-----------------+

+------------+------------------+
|RainTomorrow|     avg(Cloud9am)|
+------------+------------------+
|          No|3.9322820037105752|
|         Yes|  6.09999030161963|
+------------+------------------+



In [ ]:
# Media dell'attributo Cloud3pm (groupby RainToday e RainTomorrow)

df.groupBy("RainToday").agg({'Cloud3pm':'avg'}).show()
df.groupBy("RainTomorrow").agg({'Cloud3pm':'avg'}).show()

+---------+------------------+
|RainToday|     avg(Cloud3pm)|
+---------+------------------+
|       No| 4.086651706260156|
|      Yes|5.8270744098017735|
|     null| 5.320197044334975|
+---------+------------------+

+------------+-----------------+
|RainTomorrow|    avg(Cloud3pm)|
+------------+-----------------+
|          No|3.921895974448782|
|         Yes|6.360065063091483|
+------------+-----------------+



#### GroupBy Location

In [ ]:
# Media dell'attributo Evaporation 
# groupby Sunshine, ossia l'attributo con la correlazione più forte con Evaporation
# groupby Location

df.groupBy("Location","Sunshine").agg({'Evaporation':'avg'}).sort('Location').show(5)

+--------+--------+-----------------+
|Location|Sunshine| avg(Evaporation)|
+--------+--------+-----------------+
|Adelaide|      11|           9.3875|
|Adelaide|     3.4|              2.6|
|Adelaide|    11.2|7.999999999999999|
|Adelaide|    12.5|9.411111111111111|
|Adelaide|     7.8|5.046153846153846|
+--------+--------+-----------------+
only showing top 5 rows



In [ ]:
# Media dell'attributo Sunshine 
# groupby Evaporation, ossia l'attributo con la correlazione più forte con Sunshine
# groupby Location

df.groupBy("Location","Evaporation").agg({'Sunshine':'avg'}).sort('Location').show(5)

+--------+-----------+-----------------+
|Location|Evaporation|    avg(Sunshine)|
+--------+-----------+-----------------+
|Adelaide|       14.4|9.200000000000001|
|Adelaide|        7.8|8.322222222222223|
|Adelaide|       15.4|            11.25|
|Adelaide|       19.2|             null|
|Adelaide|       11.2|9.919999999999998|
+--------+-----------+-----------------+
only showing top 5 rows



In [ ]:
# Media dell'attributo Cloud9am 
# groupby Humidity3pm, ossia l'attributo con la correlazione più forte con Cloud9am
# groupby Location

df.groupBy("Location","Humidity3pm").agg({'Cloud9am':'avg'}).sort('Location', ascending=True).show(5)

+--------+-----------+-------------+
|Location|Humidity3pm|avg(Cloud9am)|
+--------+-----------+-------------+
|Adelaide|         86|         null|
|Adelaide|         56|         null|
|Adelaide|         41|         null|
|Adelaide|         26|         null|
|Adelaide|         91|         null|
+--------+-----------+-------------+
only showing top 5 rows



In [ ]:
# Media dell'attributo Cloud3pm 
# groupby Humidity3pm, ossia l'attributo con la correlazione più forte con Cloud3pm
# groupby Location

df.groupBy("Location","Humidity3pm").agg({'Cloud3pm':'avg'}).sort('Location', ascending=True).show(5)

+--------+-----------+-------------+
|Location|Humidity3pm|avg(Cloud3pm)|
+--------+-----------+-------------+
|Adelaide|         86|         null|
|Adelaide|         56|         null|
|Adelaide|         41|         null|
|Adelaide|         26|         null|
|Adelaide|         91|         null|
+--------+-----------+-------------+
only showing top 5 rows



Per diverse città è impossibile fare la sostituzione dei null values con il groubby perché presentano troppi null values. Raggruppiamo quindi per l'attributo con la correlazione più forte e per periodo dell'anno.

#### GroupBy Month

In [ ]:
df.groupBy("Month").count().sort("Month").show()

+-----+-----+
|Month|count|
+-----+-----+
|   01|12921|
|   02|10525|
|   03|13036|
|   04|11306|
|   05|13055|
|   06|12389|
|   07|11779|
|   08|11762|
|   09|11345|
|   10|11804|
|   11|11461|
|   12|10810|
+-----+-----+



In [ ]:
# Media dell'attributo Evaporation 
# groupby Sunshine, ossia l'attributo con la correlazione più forte con Evaporation
# groupby Month

df.groupBy("Month","Sunshine").agg({'Evaporation':'avg'}).sort('Month').show(5)

+-----+--------+------------------+
|Month|Sunshine|  avg(Evaporation)|
+-----+--------+------------------+
|   01|    10.3|   8.9438202247191|
|   01|     2.9|           6.86875|
|   01|     8.2| 8.463888888888889|
|   01|     5.9| 6.769230769230769|
|   01|    13.8|7.7700000000000005|
+-----+--------+------------------+
only showing top 5 rows



In [ ]:
# Media dell'attributo Sunshine 
# groupby Evaporation, ossia l'attributo con la correlazione più forte con Sunshine
# groupby Date

df.groupBy("Month","Evaporation").agg({'Sunshine':'avg'}).sort('Month').show(5)

+-----+-----------+------------------+
|Month|Evaporation|     avg(Sunshine)|
+-----+-----------+------------------+
|   01|       11.8|10.594871794871793|
|   01|       17.6| 9.014285714285714|
|   01|       10.8|11.287209302325582|
|   01|        8.2| 9.752100840336139|
|   01|       16.4|11.838461538461539|
+-----+-----------+------------------+
only showing top 5 rows



In [ ]:
# Media dell'attributo Cloud9am 
# groupby Humidity3pm, ossia l'attributo con la correlazione più forte con Cloud9am
# groupby Month

df.groupBy("Month","Humidity3pm").agg({'Cloud9am':'avg'}).sort('Month').show(5)

+-----+-----------+-----------------+
|Month|Humidity3pm|    avg(Cloud9am)|
+-----+-----------+-----------------+
|   01|         14|1.839080459770115|
|   01|         65|6.008196721311475|
|   01|         37|4.256410256410256|
|   01|         92|          7.28125|
|   01|         49|5.266129032258065|
+-----+-----------+-----------------+
only showing top 5 rows



In [ ]:
# Media dell'attributo Cloud3pm 
# groupby Humidity3pm, ossia l'attributo con la correlazione più forte con Cloud3pm
# groupby Month

df.groupBy("Month","Humidity3pm").agg({'Cloud3pm':'avg'}).sort('Month').show(5)

+-----+-----------+------------------+
|Month|Humidity3pm|     avg(Cloud3pm)|
+-----+-----------+------------------+
|   01|         14|2.6206896551724137|
|   01|         65| 5.404958677685951|
|   01|         37|2.9357798165137616|
|   01|         92| 7.515151515151516|
|   01|         49| 3.725663716814159|
+-----+-----------+------------------+
only showing top 5 rows



## Sostituzione missing values con mediana

In [ ]:
df = df.fillna('NA')
df.show(5)

+----------+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+-------+------------+-----+------+---------------+
|      Date|Location|MinTemp|MaxTemp|Rainfall|Evaporation|Sunshine|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|Cloud9am|Cloud3pm|Temp9am|Temp3pm|RainToday|RISK_MM|RainTomorrow|Month|Season|         Region|
+----------+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+-------+------------+-----+------+---------------+
|01/12/2008|  Albury|   13.4|   22.9|     0.6|         NA|      NA|          W|           44|         W|       WNW|          20|          24|       

In [ ]:
cols_to_impute = ["Evaporation", "Sunshine", "Pressure9am", "Pressure3pm", "Cloud9am", "Cloud3pm"]

In [ ]:
# verifichiamo se a seconda di quando sta per piovere o se sta piovendo consecutivamente
# i valori possano seguire distribuzioni diverse
df = df.withColumn( 'it-will-rain' , when( (df.RainToday == 'No') & ( df.RainTomorrow == 'Yes'), 'Yes' ).otherwise('No') )   # oggi non piove e domani si
df = df.withColumn( 'no-rain' , when( (df.RainToday == 'No') & ( df.RainTomorrow == 'No'), 'Yes' ).otherwise('No') )         # oggi non piove e neanche domani
df = df.withColumn( 'it-rain' , when( (df.RainToday == 'Yes') & ( df.RainTomorrow == 'Yes'), 'Yes' ).otherwise('No') )       # oggi piove e pure domani
df = df.withColumn( 'end-rain' , when( (df.RainToday == 'Yes') & ( df.RainTomorrow == 'No'), 'Yes' ).otherwise('No') )       # oggi non piove e neanche domani

df.show(5)

+----------+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+-------+------------+-----+------+---------------+------------+-------+-------+--------+
|      Date|Location|MinTemp|MaxTemp|Rainfall|Evaporation|Sunshine|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|Cloud9am|Cloud3pm|Temp9am|Temp3pm|RainToday|RISK_MM|RainTomorrow|Month|Season|         Region|it-will-rain|no-rain|it-rain|end-rain|
+----------+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+-------+------------+-----+------+---------------+------------+-------+-------+--------+
|01/12/2008|  Albury|   13.4|   22

In [ ]:
df_no_rain = df.where(df['no-rain']== 'Yes')
df_will_rain = df.where(df['it-will-rain'] == 'Yes')
df_it_rain = df.where(df['it-rain'] == 'Yes')
df_end_rain = df.where(df['end-rain'] == 'Yes')

In [ ]:
# questa funzione mi serve per prendere un campione dell'attributo scelto
# che verra ritornato come array di numpy
def get_sample_(df, attr):
  df_filtered=df.filter(col(attr) != 'NA')
  sample = np.array( df_filtered.select(col(attr)).rdd.takeSample(False, 3000, seed= 42) ).flatten()
  return sample

In [ ]:
def check_distribution(attr):
  # ottengo un campione dei dati dell'attributo specificato
  sample_no_rain = get_sample_(df_no_rain, attr).astype(np.float)
  sample_will_rain = get_sample_(df_will_rain, attr).astype(np.float)
  sample_rain= get_sample_(df_it_rain, attr).astype(np.float)
  sample_end_rain = get_sample_(df_end_rain, attr).astype(np.float)

  # faccio un test per vedere se seguono la stessa distribuzione
  from scipy.stats import median_test

  alfa = 0.01    # alfa e il livello di confidenza con il quale vogliamo essere sicuri se accettare o 
                # rifiutare l'ipotesi nulla

  print("Median Test for attribute <"+ attr + ">")
  stat, p, med, tbl = median_test(sample_no_rain, sample_will_rain)
  print('   no-rain and will-rain ',p < alfa)

  stat, p, med, tbl = median_test(sample_will_rain, sample_rain)
  print('   will-rain and it-rain ',p < alfa)

  stat, p, med, tbl = median_test(sample_rain, sample_end_rain)
  print('   it-rain and end-rain', p < alfa)
  print("")

In [ ]:
for attr in cols_to_impute:
  check_distribution(attr)

Median Test for attribute <Evaporation>
   no-rain and will-rain  True
   will-rain and it-rain  True
   it-rain and end-rain False

Median Test for attribute <Sunshine>
   no-rain and will-rain  True
   will-rain and it-rain  True
   it-rain and end-rain True

Median Test for attribute <Pressure9am>
   no-rain and will-rain  True
   will-rain and it-rain  True
   it-rain and end-rain True

Median Test for attribute <Pressure3pm>
   no-rain and will-rain  True
   will-rain and it-rain  True
   it-rain and end-rain True

Median Test for attribute <Cloud9am>
   no-rain and will-rain  True
   will-rain and it-rain  True
   it-rain and end-rain True

Median Test for attribute <Cloud3pm>
   no-rain and will-rain  True
   will-rain and it-rain  True
   it-rain and end-rain True



In [ ]:
def get_median(df, attr):
  # pulisco dai NA values
  df_filtered= df.filter(col(attr) != 'NA')
  # converto la colonna in float
  df_float = df_filtered.withColumn(attr, col(attr).cast(FloatType()))
  return df_float.approxQuantile(attr, [0.5], 0.25)  # ritorna la mediana dell'attributo scelto


def get_median_(df, attr, dict_median):
  # pulisco dai NA values
  df_filtered= df.filter(col(attr) != 'NA')
  # converto la colonna in float
  df_float = df_filtered.withColumn(attr, col(attr).cast(FloatType()))
  median = df_float.approxQuantile(attr, [0.5], 0.25)
  dict_median[attr] = median
  return dict_median 

In [ ]:
# sostituiamo i valori mancanti con la mediana
dict_median = {}
for attr in cols_to_impute:
  if attr!="Evaporation":
    dict_median = get_median_(df,attr, dict_median)
    median_attr = str(dict_median[attr][0])
    df = df.withColumn(attr, regexp_replace(attr, 'NA', median_attr)).withColumn(attr, col(attr).cast(FloatType()))

Poiché no-rain, will-rain e it-rain hanno dei p-value molto piccoli possiamo assumere che abbiano la stessa mediana, ma per quanto riguarda it-rain e end-rain, non seguono la stessa mediana

In [ ]:
# imputiamo quindi i valori, ricordandoci che siccome
# no-rain, will-rain e it-rain hanno la stessa mediana, per loro 
# sara identica, mentre lo stesso non avverra per end-rain

median_rain = get_median(df.where(col('end-rain') == 'No' ), 'Evaporation')
median_rain= str(median_rain[0])

median_noRain = get_median(df.where(col('end-rain') == 'Yes'), 'Evaporation')
median_noRain= str(median_noRain[0])

# sostituiamo i valori mancanti con la mediana
df = df.withColumn('Evaporation', when( (col('end-rain') == 'No') & (col('Evaporation') == 'NA') , regexp_replace('Evaporation', 'NA', median_rain))\
                            .otherwise( when(   (col('end-rain') == 'Yes') & (col('Evaporation') == 'NA') , regexp_replace('Evaporation', 'NA', median_noRain))\
                            .otherwise( col('Evaporation')))).withColumn('Evaporation', col('Evaporation').cast(FloatType()))

In [ ]:
for attr in cols_to_impute:
  df.select([count(when(isnan(attr),True))]).show()

+-------------------------------------------------+
|count(CASE WHEN isnan(Evaporation) THEN true END)|
+-------------------------------------------------+
|                                                0|
+-------------------------------------------------+

+----------------------------------------------+
|count(CASE WHEN isnan(Sunshine) THEN true END)|
+----------------------------------------------+
|                                             0|
+----------------------------------------------+

+-------------------------------------------------+
|count(CASE WHEN isnan(Pressure9am) THEN true END)|
+-------------------------------------------------+
|                                                0|
+-------------------------------------------------+

+-------------------------------------------------+
|count(CASE WHEN isnan(Pressure3pm) THEN true END)|
+-------------------------------------------------+
|                                                0|
+-----------------------

In [ ]:
df = df.drop("it-will-rain", "no-rain", "it-rain", "end-rain")
df.show(10)

+----------+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+-------+------------+-----+------+---------------+
|      Date|Location|MinTemp|MaxTemp|Rainfall|Evaporation|Sunshine|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|Cloud9am|Cloud3pm|Temp9am|Temp3pm|RainToday|RISK_MM|RainTomorrow|Month|Season|         Region|
+----------+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+-------+------------+-----+------+---------------+
|01/12/2008|  Albury|   13.4|   22.9|     0.6|        3.2|     5.6|          W|           44|         W|       WNW|          20|          24|       

In [ ]:
cat_cols = ["Date", "Location", "WindGustDir", "WindDir9am", "WindDir3pm",
            "RainTomorrow", "RainToday", "Evaporation", "Sunshine", "Pressure9am",
            "Pressure3pm", "Cloud9am", "Cloud3pm", 'Month', 'Season', 'Region']

col_to_float = [i for i in df.columns if i not in cat_cols]

In [ ]:
col_to_float

['MinTemp',
 'MaxTemp',
 'Rainfall',
 'WindGustSpeed',
 'WindSpeed9am',
 'WindSpeed3pm',
 'Humidity9am',
 'Humidity3pm',
 'Temp9am',
 'Temp3pm',
 'RISK_MM']

In [ ]:
for attr in col_to_float:
  df = df.withColumn(attr, col(attr).cast(FloatType()))

In [ ]:
df.show()

+----------+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+-------+------------+-----+------+---------------+
|      Date|Location|MinTemp|MaxTemp|Rainfall|Evaporation|Sunshine|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|Cloud9am|Cloud3pm|Temp9am|Temp3pm|RainToday|RISK_MM|RainTomorrow|Month|Season|         Region|
+----------+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+-------+------------+-----+------+---------------+
|01/12/2008|  Albury|   13.4|   22.9|     0.6|        3.2|     5.6|          W|         44.0|         W|       WNW|        20.0|        24.0|       

In [ ]:
df

DataFrame[Date: string, Location: string, MinTemp: float, MaxTemp: float, Rainfall: float, Evaporation: float, Sunshine: float, WindGustDir: string, WindGustSpeed: float, WindDir9am: string, WindDir3pm: string, WindSpeed9am: float, WindSpeed3pm: float, Humidity9am: float, Humidity3pm: float, Pressure9am: float, Pressure3pm: float, Cloud9am: float, Cloud3pm: float, Temp9am: float, Temp3pm: float, RainToday: string, RISK_MM: float, RainTomorrow: string, Month: string, Season: string, Region: string]

In [ ]:
# Salvataggio nuovo dataframe 'rain_tomorrow_australia'

df.toPandas().to_csv('drive/MyDrive/DDAM/Australia Rain/rain_tomorrow_australia_PROVA.csv', header=True)

## Confronto tra mediana e regressione per la sostituzione dei missing values

In [ ]:
# Caricamento nuovo dataset senza missing values

filename = 'drive/MyDrive/DDAM/Australia Rain/rain_tomorrow_australia.csv'
df_median = sqlCtx.read.load(filename, format="csv", sep=",", inferSchema="true", header="true")

# Caricamento dataset originale

filename2 = 'drive/MyDrive/DDAM/Australia Rain/australia_rain_tomorrow_raw.csv'
df_original = sqlCtx.read.load(filename2, format="csv", sep=",", inferSchema="true", header="true")

Prendo un campione di osservazioni del dataframe. Per capire quali righe non avevano missing values utilizzo direttamente il vecchio dataframe.

In [ ]:
# Creo una colonna id nel dataframe originale
df_original = df_original.select("*").withColumn("_c0", monotonically_increasing_id())

# Riordino le colonne
df_original = df_original.select('_c0','Date','Location','MinTemp','MaxTemp','Rainfall','Evaporation','Sunshine',
 'WindGustDir','WindGustSpeed','WindDir9am','WindDir3pm','WindSpeed9am','WindSpeed3pm','Humidity9am',
 'Humidity3pm','Pressure9am','Pressure3pm','Cloud9am','Cloud3pm','Temp9am','Temp3pm','RainToday','RISK_MM',
 'RainTomorrow') 

Nel nuovo dataframe sono stati sostituiti i missing values per le colonne Evaporation, Sunshine, Cloud9am, Cloud3pm, Pressure9am, Pressure3pm. 

Per fare la predizione con il vecchio dataframe devo escludere tutte queste colonne (se i valori sono stati sostituiti è perché erano missing values -> impossibile fare regressione).

Tengo tuttavia tutti i valori di Sunshine, per cui devo fare la predizione.

In [ ]:
cols = ['_c0','Sunshine','MinTemp','MaxTemp','Rainfall','WindGustSpeed','WindSpeed9am',
        'WindSpeed3pm','Humidity9am','Humidity3pm','Temp3pm']

df_original = df_original.select(*cols)
df_median = df_median.select(*cols)

In [ ]:
# Rimuovo osservazioni con missing values (ad esclusione di Sunshine) dal df originale

df_original = df_original.replace('NA', None)
df_original = df_original.na.drop(subset=['_c0','MinTemp','MaxTemp','Rainfall','WindGustSpeed','WindSpeed9am','WindSpeed3pm','Humidity9am','Humidity3pm','Temp3pm'])

In [ ]:
print('Totale osservazioni:', df_original.count())
print()
print('Null values per column:')

Dict_Null = {col:df_original.filter(df_original[col].isNull()).count() for col in df_original.columns}
Dict_Null = sorted(Dict_Null.items(), key=lambda x: x[1], reverse=True)

for i in Dict_Null:
	print(i[0], i[1])

Totale osservazioni: 129246

Null values per column:
Sunshine 60138
_c0 0
MinTemp 0
MaxTemp 0
Rainfall 0
WindGustSpeed 0
WindSpeed9am 0
WindSpeed3pm 0
Humidity9am 0
Humidity3pm 0
Temp3pm 0


In [ ]:
# Del nuovo dataset tengo solo le righe che nel vecchio dataset non avevano null values (escluso Sunshine)
# Faccio una join sulla colonna id

df_median = df_median.join(df_original, df_median._c0 == df_original._c0, "semi")

df_median.join(df_original, df_median._c0 == df_original._c0, 'semi').show(5)
df_median.join(df_original, df_median._c0 == df_original._c0, 'semi').count()

df_median.filter(df_median['Sunshine'].isNull()).count()

print('N° osservazioni rimaste', df_median.count())

+---+--------+-------+-------+--------+-------------+------------+------------+-----------+-----------+-------+
|_c0|Sunshine|MinTemp|MaxTemp|Rainfall|WindGustSpeed|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Temp3pm|
+---+--------+-------+-------+--------+-------------+------------+------------+-----------+-----------+-------+
|  0|     5.6|   13.4|   22.9|     0.6|         44.0|        20.0|        24.0|       71.0|       22.0|   21.8|
|  1|     5.6|    7.4|   25.1|     0.0|         44.0|         4.0|        22.0|       44.0|       25.0|   24.3|
|  2|     5.6|   12.9|   25.7|     0.0|         46.0|        19.0|        26.0|       38.0|       30.0|   23.2|
|  3|     5.6|    9.2|   28.0|     0.0|         24.0|        11.0|         9.0|       45.0|       16.0|   26.5|
|  4|     5.6|   17.5|   32.3|     1.0|         41.0|         7.0|        20.0|       82.0|       33.0|   29.7|
+---+--------+-------+-------+--------+-------------+------------+------------+-----------+-----------+-

#### Regressione per Sunshine

Faccio la regressione per Sunshine. Ottengo una colonna di predizioni e la confronto con la colonna Sunshine del nuovo dataset.

In [ ]:
# Seleziono colonne
df_regression = df_median

# Trasformo in float
df_regression = df_regression.select(*(col(c).cast("float").alias(c) for c in df_regression.columns))

In [ ]:
df_regression.printSchema()

root
 |-- _c0: float (nullable = true)
 |-- Sunshine: float (nullable = true)
 |-- MinTemp: float (nullable = true)
 |-- MaxTemp: float (nullable = true)
 |-- Rainfall: float (nullable = true)
 |-- WindGustSpeed: float (nullable = true)
 |-- WindSpeed9am: float (nullable = true)
 |-- WindSpeed3pm: float (nullable = true)
 |-- Humidity9am: float (nullable = true)
 |-- Humidity3pm: float (nullable = true)
 |-- Temp3pm: float (nullable = true)



In [ ]:
# Creo il vettore di valori

in_cols= [col for col in df_regression.columns if col != 'Sunshine']

vecAssembler = VectorAssembler(inputCols=in_cols, outputCol="features")
vec_df = vecAssembler.transform(df_regression)
vec_df = vec_df.select(['features', 'Sunshine'])
vec_df.show(3)

+--------------------+--------+
|            features|Sunshine|
+--------------------+--------+
|[0.0,13.399999618...|     5.6|
|[1.0,7.4000000953...|     5.6|
|[2.0,12.899999618...|     5.6|
+--------------------+--------+
only showing top 3 rows



In [ ]:
vec_df.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Sunshine: float (nullable = true)



In [ ]:
# Creo modello e ottengo le predizioni

lr = LinearRegression(featuresCol = 'features', labelCol='Sunshine')
lr_model = lr.fit(vec_df)
lr_predictions = lr_model.transform(vec_df)

print('N° predizioni:',lr_predictions.count())

N° predizioni: 84525


In [ ]:
lr_predictions.show(5)
lr_predictions.printSchema()

+--------------------+--------+------------------+
|            features|Sunshine|        prediction|
+--------------------+--------+------------------+
|[0.0,13.399999618...|     5.6|7.0110699313355855|
|[1.0,7.4000000953...|     5.6| 7.677187066329452|
|[2.0,12.899999618...|     5.6| 7.607937954908657|
|[3.0,9.1999998092...|     5.6|  8.30139187127823|
|[4.0,17.5,32.2999...|     5.6| 6.887682983888796|
+--------------------+--------+------------------+
only showing top 5 rows

root
 |-- features: vector (nullable = true)
 |-- Sunshine: float (nullable = true)
 |-- prediction: double (nullable = false)



#### Evaluation

Ora abbiamo la colonna Sunshine e la colonna con le predizioni. Misuriamo quanto sono diverse.

In [ ]:
lr_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="Sunshine",metricName="r2")
lr_evaluator2 = RegressionEvaluator(predictionCol="prediction", labelCol="Sunshine",metricName="rmse")

print("R Squared = %g" % lr_evaluator.evaluate(lr_predictions))
print("RMSE = %g" % lr_evaluator2.evaluate(lr_predictions))

R Squared = 0.213008
RMSE = 2.50081


In [ ]:
# Aggiungo la colonna differenza (in valore assoluto)
from  pyspark.sql.functions import abs

lr_predictions = lr_predictions.withColumn('difference', abs(lr_predictions.Sunshine - lr_predictions.prediction))

lr_predictions.show(30)

+--------------------+--------+------------------+------------------+
|            features|Sunshine|        prediction|        difference|
+--------------------+--------+------------------+------------------+
|[0.0,13.399999618...|     5.6|7.0110699313355855|1.4110700267030172|
|[1.0,7.4000000953...|     5.6| 7.677187066329452|2.0771871616968838|
|[2.0,12.899999618...|     5.6| 7.607937954908657|2.0079380502760884|
|[3.0,9.1999998092...|     5.6|  8.30139187127823|2.7013919666456623|
|[4.0,17.5,32.2999...|     5.6| 6.887682983888796|1.2876830792562277|
|[5.0,14.600000381...|     5.6|7.5478286624702156|1.9478287578376472|
|[6.0,14.300000190...|     5.6|7.5179253184253865|1.9179254137928181|
|[7.0,7.6999998092...|     5.6|  8.02389268798429|2.4238927833517216|
|[8.0,9.6999998092...|     5.6| 7.839619773369929|2.2396198687373605|
|[9.0,13.100000381...|     5.6|7.7269787796136065| 2.126978874981038|
|[10.0,13.39999961...|     5.6|  7.86187935215613|2.2618794475235617|
|[11.0,15.89999961..

In [ ]:
lr_predictions.select(lr_predictions['difference']).summary().show()

+-------+--------------------+
|summary|          difference|
+-------+--------------------+
|  count|               84525|
|   mean|  1.9895800645913424|
| stddev|  1.5151382702108818|
|    min|3.129227160947323...|
|    25%|  0.7564876457468417|
|    50%|   1.651940445529628|
|    75%|  2.9373357384098977|
|    max|   9.208369080744045|
+-------+--------------------+

